In [104]:
import alpaca_trade_api as tradeapi
from datetime import date
import time
import datetime
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
import pandas_ta as ta
import yfinance as yf

In [51]:
BASE_URL = 'https://paper-api.alpaca.markets'
PUB_KEY = 'PK92I7HHN7R3TNHJSJG8'
SEC_KEY = 'To6M3zJRMYfaBriBAtPru9E2uSg7MnwPLiy5CO4G'
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL)

In [156]:
def getDB(ticker):
    tick = ticker
    # Load data
    data = yf.Ticker(tick)
    df = data.history(period="1mo", interval="15m")
#     df = data.history(start="2018-12-01", end="2020-03-01")
#     start="2017-01-01", end="2017-04-30"
    
    # add data points
    df['close_per1'] = df.ta.percent_return(1)*100
    df['sma10'] = df.ta.sma(length=10)
    df['williams'] = df.ta.willr(7)


    df = df[[
            'open','close','sma10','williams'
            ]]

    df = df.dropna()
    df.reset_index(inplace=True)
    
    return df

In [157]:
data = getDB("FAS")
print(data.head())
print(data.tail())
print(len(data))
# print((df.iloc[0]['datetime']-df.iloc[0]['datetime']).minutes)
df.iloc[0]['datetime'].minute
(df.iloc[11]['datetime']-df.iloc[8]['datetime']).seconds

                   datetime        open       close       sma10   williams
0 2021-12-06 11:45:00-05:00  127.820000  127.239998  127.077421 -47.862760
1 2021-12-06 12:00:00-05:00  127.144997  127.779999  127.306921 -36.958345
2 2021-12-06 12:15:00-05:00  127.608002  128.279999  127.563921 -21.812890
3 2021-12-06 12:30:00-05:00  127.989998  127.879997  127.735920 -40.306188
4 2021-12-06 12:45:00-05:00  127.855003  128.190002  127.905930 -23.710963
                     datetime        open       close       sma10   williams
507 2022-01-03 15:00:00-05:00  134.410004  133.820007  134.492133 -99.121950
508 2022-01-03 15:15:00-05:00  133.869995  134.270004  134.473132 -56.755791
509 2022-01-03 15:30:00-05:00  134.369995  133.970001  134.403003 -83.236142
510 2022-01-03 15:45:00-05:00  134.059998  134.660004  134.382004 -24.843137
511 2022-01-03 16:00:00-05:00  134.539993  134.539993  134.419003 -35.124183
512


2700

In [158]:
# tickers = ["FAS","TNA","SOXL","TECL","LABU","YINN","DPST","NAIL"]
# tickers = ["FAS","SOXL"]
# tickers = ["SOXL","SPXL","TECL","FAS","UPRO","FNGU","TNA","LABU","YINN","DPST","NAIL","DFEN","CURE","EURL","DUSL","MIDU","EDU","MEXX","PILL","DRN","RETL","TPOR","UTSL","KORU","OILU"]
tickers = ["SOXL","FAS","FNGU","TNA","LABU","YINN","DPST","NAIL","DFEN","DUSL","MIDU","MEXX","PILL","RETL","TPOR","KORU"]

ALL = []; amount = []
for ticker in tickers:
    print()
    print("ticker: {}".format(ticker))
#     df = getDF(ticker,120)
    df = getDB(ticker)
    ## Williams trading idea ##
    returnP1 = []; returnP2 = []
    buyState = False
    # setting the outputs in the df
    for i in range(len(df)):
        if df.iloc[i]['williams'] < -90:
            buy = df.iloc[i]['close']
            buyState = True
            buyD = df.iloc[i]['datetime']

        if buyState == True and df.iloc[i]['williams'] > -10:
            sell = df.iloc[i]['close']
            R = ((sell-buy)/abs(buy))*100
            r = round(R, 2)
            returnP1.append(r)
            buyState = False
            sellD = df.iloc[i]['datetime']
            delta = round(((sellD - buyD).seconds/60),2)
            print("Bought on {} & sold on {}. Delta {} minutes. {}%".format(buyD.strftime('%Y-%m-%d'), sellD.strftime('%Y-%m-%d'), delta, r))

#     print()
#     print()
    avgP1 = np.average(returnP1)
    # avgP2 = np.average(returnP2)
    # print(avgP1)
#     print("ticker: {}".format(ticker))
#     print("Number of trade days: %d" % (len(df)))
    print("Number of trades: %d" % (len(returnP1)))
    print("Percentage average when trading williams method: {}%".format(round(avgP1,2)))
    # print("Average return if loss limit is 0.1 percent: %f" % (avgP2))
#     print(returnP1)
    ALL.append(avgP1)
    amount.append(len(returnP1))
    time.sleep(3)

print()
print()
print("total average: {}%".format(round(np.average(ALL),2)))
print("total trades: {}".format(int(np.sum(amount))))
print("with {} tickers".format(len(tickers)))


ticker: FAS
Bought on 2021-12-06 & sold on 2021-12-07. Delta 1095.0 minutes. 4.23%
Bought on 2021-12-07 & sold on 2021-12-07. Delta 60.0 minutes. 0.99%
Bought on 2021-12-08 & sold on 2021-12-08. Delta 270.0 minutes. 1.72%
Bought on 2021-12-09 & sold on 2021-12-09. Delta 150.0 minutes. 1.67%
Bought on 2021-12-10 & sold on 2021-12-10. Delta 60.0 minutes. 1.21%
Bought on 2021-12-13 & sold on 2021-12-13. Delta 45.0 minutes. 1.34%
Bought on 2021-12-14 & sold on 2021-12-14. Delta 90.0 minutes. 1.5%
Bought on 2021-12-15 & sold on 2021-12-15. Delta 135.0 minutes. 1.96%
Bought on 2021-12-17 & sold on 2021-12-17. Delta 75.0 minutes. 3.93%
Bought on 2021-12-20 & sold on 2021-12-20. Delta 150.0 minutes. 1.58%
Bought on 2021-12-21 & sold on 2021-12-22. Delta 1095.0 minutes. 1.02%
Bought on 2021-12-22 & sold on 2021-12-22. Delta 75.0 minutes. 0.65%
Bought on 2021-12-23 & sold on 2021-12-23. Delta 75.0 minutes. 0.26%
Bought on 2021-12-23 & sold on 2021-12-27. Delta 1140.0 minutes. 1.53%
Bought on 20